In [2]:
# === Embodied Game of Life / HighLife + Diffusive Food (open-ended, vectorized, Pygame) ===
# This version adds:
#   • Rule mode: "normal" (Life, B3/S23) or "high" (HighLife, B36/S23)
#   • gate_every: apply food-based gates only every k ticks (k>=1). Other ticks run pure rule updates.
#
# Everything else remains as in your last configuration:
# - Pure diffusion (correct sign), no advection. Consumption on live cells (+ tiny leak).
# - Gates affect ONLY currently-alive cells (no spontaneous births).
# - Sparse, binary food initialization and per-cell rare spawning (sets to 1, idempotent).
# - Central D×D seeding draws randomness only in-window.
# - Strict synchrony: fixed dt; CFL asserted; no file saves.

import numpy as np
import pygame
import sys

# -----------------------------
# 0) Hyperparameters (edit here)
# -----------------------------
CFG = dict(
    # Rule mode
    mode="normal",          # "normal" -> Life (B3/S23) | "high" -> HighLife (B36/S23)

    # Gate cadence
    gate_every=2000,           # apply gates every k ticks (k>=1). 1 = every tick; 3 = every 3rd tick, etc.

    # Grid & timing
    H=400, W=600,
    dt=0.5,                 # fixed; CFL asserted, not adjusted
    ticks_per_render=10,    # do multiple ticks per frame for speed
    target_fps=60,

    # Food scale & init/spawn (sparse, binary)
    F_min=0.0, F_max=1.0,
    F_init_prob=1,        # initial probability per cell to have food=1 (as provided)
    food_spawn_prob=3e-5,   # per-cell per-tick probability to set food=1 (VERY rare)
    food_spawn_value=1.0,   # value written on spawn (no effect if already 1)

    # Diffusion (no advection)
    D_dead=0.0001, D_alive=0.5,
    use_harmonic_faces=True,
    enforce_cfl_assert=True,

    # Consumption + tiny global leak
    mu=0.05,                 # multiplicative uptake on live cells
    lambda_leak=3e-5,       # tiny global leak to avoid slow drift upward

    # Soft gates (tiny, local, one-sided) — ONLY for currently-alive cells
    T1=0.1,                 # scarcity threshold
    T2=0.98,                # abundance threshold
    eps_d=1,            # extra death prob on would-survive ALIVE cells
    eps_r=0.0000,          # rescue prob on would-die ALIVE cells
    alpha=50.0,             # sigmoid sharpness
    gate_on_local_meanF=True,  # use 3×3 mean of F for gating

    # Central seeding (in-window only)
    seed_zone_D=20,         # set 0 to disable
    p_seed_base=1e-1,       # base seeding prob per site per tick (in window)
    warmup_ticks=200,       # during warmup use higher p_seed to bootstrap life
    p_seed_warmup=3e-3,

    # Visualization
    cell_px=2,
    color_live=(255, 255, 255),  # white live cells
    show_food_overlay=True,

    # Logging (console prints only; no files)
    log_every_ticks=60,
)

# -----------------------------
# 1) Utilities & kernels
# -----------------------------
rng = np.random.default_rng()  # NO fixed seed → non-deterministic

H, W = CFG['H'], CFG['W']
dt = CFG['dt']

def neigh8_sum(C: np.ndarray) -> np.ndarray:
    """8-neighbor sum with periodic wrap. C is uint8 {0,1}; returns int16 counts 0..8."""
    up    = np.roll(C, -1, axis=0)
    down  = np.roll(C,  1, axis=0)
    left  = np.roll(C,  1, axis=1)
    right = np.roll(C, -1, axis=1)
    up_l    = np.roll(up,    1, axis=1)
    up_r    = np.roll(up,   -1, axis=1)
    down_l  = np.roll(down,  1, axis=1)
    down_r  = np.roll(down, -1, axis=1)
    N8 = (up + down + left + right + up_l + up_r + down_l + down_r).astype(np.int16)
    return N8

def mean3x3(X: np.ndarray) -> np.ndarray:
    """3×3 mean via shifts with periodic wrap."""
    up    = np.roll(X, -1, axis=0)
    down  = np.roll(X,  1, axis=0)
    left  = np.roll(X,  1, axis=1)
    right = np.roll(X, -1, axis=1)
    up_l    = np.roll(up,    1, axis=1)
    up_r    = np.roll(up,   -1, axis=1)
    down_l  = np.roll(down,  1, axis=1)
    down_r  = np.roll(down, -1, axis=1)
    s = (X + up + down + left + right + up_l + up_r + down_l + down_r).astype(np.float32)
    return s * (1.0/9.0)

def hmean(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    """Harmonic mean for face coefficients."""
    return (2.0 * a * b) / (a + b + 1e-12)

def sigmoid(x: np.ndarray) -> np.ndarray:
    x = x.astype(np.float32, copy=False)
    return 1.0 / (1.0 + np.exp(-x))

def build_rule_LUTs(mode: str) -> tuple[np.ndarray, np.ndarray]:
    """
    Build GoL-like LUTs:
      alive_lut[k] = 1 if an ALIVE cell with k neighbors survives; else 0.
      dead_lut[k]  = 1 if a DEAD cell with k neighbors is born;  else 0.
    Modes:
      - "normal": Life (B3/S23)
      - "high":   HighLife (B36/S23)
    """
    mode = (mode or "normal").lower().strip()
    if mode not in ("normal", "high"):
        mode = "normal"

    # Survivors S23 for both Life and HighLife
    S = {2, 3}
    if mode == "normal":
        B = {3}
    else:  # "high" -> HighLife
        B = {3, 6}

    alive_lut = np.zeros(9, dtype=np.uint8)
    dead_lut  = np.zeros(9, dtype=np.uint8)
    for k in range(9):
        alive_lut[k] = 1 if k in S else 0
        dead_lut[k]  = 1 if k in B else 0
    return alive_lut, dead_lut

alive_lut, dead_lut = build_rule_LUTs(CFG["mode"])

# Center seeding slice (optional)
if CFG['seed_zone_D'] > 0:
    D = CFG['seed_zone_D']
    r0 = (H - D) // 2
    c0 = (W - D) // 2
    r1, c1 = r0 + D, c0 + D
    def center_view(arr: np.ndarray):
        return arr[r0:r1, c0:c1]
else:
    D = 0
    r0 = c0 = r1 = c1 = 0
    def center_view(arr: np.ndarray):
        return arr[0:0, 0:0]  # empty view

# -----------------------------
# 2) Initialization
# -----------------------------
# Life grid: start empty (central seeding will bootstrap)
C = np.zeros((H, W), dtype=np.uint8)

# Food field: binary sparse initialization in {0,1}
F = (rng.random((H, W)) < CFG['F_init_prob']).astype(np.float32) * CFG['food_spawn_value']
F = np.clip(F, CFG['F_min'], CFG['F_max']).astype(np.float32)

# -----------------------------
# 3) Pygame setup
# -----------------------------
pygame.init()
pygame.display.set_caption("Embodied Life/HighLife + Diffusive Food (Open-Ended)")
scale = CFG['cell_px']
screen = pygame.display.set_mode((W*scale, H*scale))
clock = pygame.time.Clock()
font = pygame.font.SysFont(None, 20)

paused = False
tick = 0

def assert_cfl(Dx: np.ndarray, Dy: np.ndarray, dt: float):
    if not CFG['enforce_cfl_assert']:
        return
    # Stability for explicit variable-D diffusion on 2D grid with unit spacing: dt <= 1/(4*Dmax_face)
    D_face_max = float(max(Dx.max(initial=0.0), Dy.max(initial=0.0)))
    if D_face_max <= 0.0:
        return
    limit = 1.0 / (4.0 * D_face_max)
    if dt > limit + 1e-12:
        raise RuntimeError(f"CFL violated: dt={dt:.6f} > 1/(4*Dmax)={limit:.6f}. "
                           f"Decrease dt or D_alive. (Dmax_face={D_face_max:.6f})")

def build_frame_surface(C: np.ndarray, F: np.ndarray) -> pygame.Surface:
    # Base food as green channel
    G = (np.clip(F, 0.0, 1.0) * 255.0).astype(np.uint8)
    rgb = np.zeros((H, W, 3), dtype=np.uint8)
    if CFG['show_food_overlay']:
        rgb[..., 1] = G  # green
    # Overlay live cells as white
    live = (C == 1)
    rgb[live] = CFG['color_live']
    # Pygame expects (width, height, channels) → transpose
    surf = pygame.surfarray.make_surface(np.transpose(rgb, (1, 0, 2)))
    if scale != 1:
        surf = pygame.transform.scale(surf, (W*scale, H*scale))
    return surf

# -----------------------------
# 4) Simulation step (one tick)
# -----------------------------
def step_once(C: np.ndarray, F: np.ndarray, tick: int) -> tuple[np.ndarray, np.ndarray, dict]:
    # 1) Rule proposal (strict uint8 arithmetic via LUTs)
    N8 = neigh8_sum(C)                                    # int16
    C_star = (alive_lut[N8] * C + dead_lut[N8] * (1 - C)).astype(np.uint8)  # 0/1

    # 2) Food-based gates — ONLY on currently-alive cells — applied every k ticks
    do_gate = (CFG['gate_every'] >= 1) and ((tick % int(CFG['gate_every'])) == 0)

    if do_gate:
        Fgate = mean3x3(F) if CFG['gate_on_local_meanF'] else F
        phi_d = sigmoid(CFG['alpha'] * (CFG['T1'] - Fgate)).astype(np.float32)
        phi_r = sigmoid(CFG['alpha'] * (Fgate - CFG['T2'])).astype(np.float32)

        U = rng.random((H, W), dtype=np.float32)  # randomness for gates

        # Masks (bool), combine as uint8 only when constructing C_new
        alive_now     = (C == 1)
        would_survive = alive_now & (C_star == 1)  # alive -> alive (rule)
        would_die     = alive_now & (C_star == 0)  # alive -> dead  (rule)
        born_mask     = (C == 0) & (C_star == 1)   # dead -> alive  (rule birth)

        keep_surv = (U < (1.0 - CFG['eps_d'] * phi_d)).astype(np.uint8) * would_survive.astype(np.uint8)
        rescue    = (U < (CFG['eps_r'] * phi_r)).astype(np.uint8)       * would_die.astype(np.uint8)
        born      = born_mask.astype(np.uint8)  # accept births strictly from rule

        C_new = keep_surv + rescue + born
        C_new = np.minimum(C_new, 1).astype(np.uint8)  # ensure binary
    else:
        # No gates this tick: use pure rule outcome
        C_new = C_star

    # 3) Central seeding (in-window only; affects only dead→alive in the window)
    if CFG['seed_zone_D'] > 0:
        p_seed = CFG['p_seed_warmup'] if tick < CFG['warmup_ticks'] else CFG['p_seed_base']
        if p_seed > 0.0:
            Cc = center_view(C_new)
            Uc = rng.random(Cc.shape, dtype=np.float32)
            seeds = (Uc < p_seed).astype(np.uint8)
            Cc |= seeds  # in-place OR keeps values in {0,1}

    # 4) Food spawning (per-cell very rare Bernoulli), BEFORE diffusion
    if CFG['food_spawn_prob'] > 0.0:
        Us = rng.random((H, W), dtype=np.float32)
        spawn_mask = (Us < CFG['food_spawn_prob'])
        if spawn_mask.any():
            F = np.where(spawn_mask, CFG['food_spawn_value'], F)

    # 5) Diffusion (conservative transport) — CORRECT SIGN
    rho = mean3x3(C_new).astype(np.float32)  # [0,1]
    D_cell = (CFG['D_dead'] + (CFG['D_alive'] - CFG['D_dead']) * rho).astype(np.float32)

    # Face diffusivities (harmonic mean)
    D_right = np.roll(D_cell, -1, axis=1)
    D_down  = np.roll(D_cell, -1, axis=0)
    Dx = hmean(D_cell, D_right)  # faces between (i,j) and (i,j+1)
    Dy = hmean(D_cell, D_down)   # faces between (i,j) and (i+1,j)

    # CFL assert (fixed dt; do NOT change dt)
    if CFG['enforce_cfl_assert']:
        D_face_max = float(max(Dx.max(initial=0.0), Dy.max(initial=0.0)))
        if D_face_max > 0.0:
            limit = 1.0 / (4.0 * D_face_max)
            if dt > limit + 1e-12:
                raise RuntimeError(f"CFL violated: dt={dt:.6f} > 1/(4*Dmax)={limit:.6f}. "
                                   f"Decrease dt or D_alive. (Dmax_face={D_face_max:.6f})")

    # Fluxes (Fick: J = -D ∇F)
    F_right = np.roll(F, -1, axis=1)
    F_down  = np.roll(F, -1, axis=0)
    Jx = -Dx * (F_right - F)   # horizontal face fluxes
    Jy = -Dy * (F_down  - F)   # vertical face fluxes

    # Divergence: outflow - inflow using periodic wrap
    Jx_left = np.roll(Jx,  1, axis=1)
    Jy_up   = np.roll(Jy,  1, axis=0)
    divJ = (Jx - Jx_left) + (Jy - Jy_up)

    # CORRECT update: F_next = F - dt * divJ  (transport only; mass-conservative on torus)
    F_tilde = (F - dt * divJ).astype(np.float32)

    # 6) Consumption + tiny leak, then clamp
    F_new = F_tilde * np.exp(-CFG['mu'] * C_new * dt, dtype=np.float32) - CFG['lambda_leak'] * F_tilde * dt
    F_new = np.clip(F_new.astype(np.float32), CFG['F_min'], CFG['F_max'])

    # Diagnostics (in-memory only)
    flip_rate = float(np.mean(C_new != C_star))
    diag = dict(flip_rate=flip_rate,
                live_mass=int(C_new.sum()),
                total_food=float(F_new.sum()))

    return C_new, F_new, diag

# -----------------------------
# 5) Main loop
# -----------------------------
paused = False
tick = 0
running = True
diag_last = None

while running:
    # Handle events
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
            elif event.key == pygame.K_SPACE:
                paused = not paused

    if not paused:
        # Do several ticks between renders
        for _ in range(CFG['ticks_per_render']):
            C, F, diag_last = step_once(C, F, tick)
            tick += 1
            if (tick % CFG['log_every_ticks'] == 0) and diag_last is not None:
                print(f"[t={tick:6d}] live={diag_last['live_mass']:7d} | "
                      f"food={diag_last['total_food']:.1f} | flips={diag_last['flip_rate']*100:.3f}%")

    # Render
    frame = build_frame_surface(C, F)
    screen.blit(frame, (0, 0))

    # Small HUD
    if diag_last is not None:
        hud = f"mode={CFG['mode']}  gate_every={CFG['gate_every']}  t={tick}  live={diag_last['live_mass']}  food={diag_last['total_food']:.0f}  flips={diag_last['flip_rate']*100:.2f}%  {'PAUSED' if paused else ''}"
        text = font.render(hud, True, (255, 255, 255))
        screen.blit(text, (8, 8))

    pygame.display.flip()
    clock.tick(CFG['target_fps'])

pygame.quit()
try:
    sys.exit(0)
except SystemExit:
    pass


[t=    60] live=      1 | food=239782.3 | flips=0.000%
[t=   120] live=      1 | food=239565.7 | flips=0.000%
[t=   180] live=      4 | food=239349.8 | flips=0.000%
[t=   240] live=    217 | food=238983.6 | flips=0.000%
[t=   300] live=    300 | food=238494.2 | flips=0.000%
[t=   360] live=    389 | food=238031.7 | flips=0.000%
[t=   420] live=    319 | food=237609.5 | flips=0.000%
[t=   480] live=    327 | food=237226.5 | flips=0.000%
[t=   540] live=    373 | food=236866.0 | flips=0.000%
[t=   600] live=    379 | food=236478.4 | flips=0.000%
[t=   660] live=    430 | food=236074.7 | flips=0.000%
[t=   720] live=    474 | food=235688.9 | flips=0.000%
[t=   780] live=    519 | food=235285.8 | flips=0.000%
[t=   840] live=    494 | food=234843.4 | flips=0.000%
[t=   900] live=    562 | food=234368.1 | flips=0.000%
[t=   960] live=    548 | food=233849.6 | flips=0.000%
[t=  1020] live=    638 | food=233373.8 | flips=0.000%
[t=  1080] live=    616 | food=232933.3 | flips=0.000%
[t=  1140]